In [2]:
# @title Setup
!pip install sodapy
!pip install unidecode
import pandas as pd
import requests
from sodapy import Socrata
from google.colab import drive
import os
import time
from IPython.display import display, Markdown
from google.colab import drive
from unidecode import unidecode
from requests.exceptions import ReadTimeout

client = Socrata("www.datos.gov.co", None)
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 3.7 MB/s eta 0:00:00


Mounted at /content/drive


In [ ]:
def download_and_save_data_chunks_departamento(resource_id, filename,departamento,tipo,chunk_size=500000, limit=None):
    chunk_number = 1
    path = "/content/drive/My Drive/DANE/chunks/"+tipo+"/"
    archive = filename.split('/')[-1][:-4]
    while os.path.exists(f'{path}{archive}/{archive}_chunk_{chunk_number}.csv'):
        chunk_number += 1
    if chunk_number > 1:
        print(f'Resuming from chunk {chunk_number} on {archive}')
        offset = (chunk_number - 1) * chunk_size
    else:
        offset = 0
    if limit is None:
        metadata = client.get_metadata(resource_id)
        limit = int(metadata['columns'][0]['cachedContents']['largest'])
    while offset < limit:
        results = client.get(resource_id,where=f"departamento='{departamento}'", limit=chunk_size, offset=offset)
        if not results:
            print(f'❌ No existen más datos para{archive}')
            break
        chunk_filename = f'{path}{archive}/{archive}_chunk_{chunk_number}.csv'
        pd.DataFrame.from_records(results).to_csv(chunk_filename, index=False)
        print(f'✅ Downloaded and saved chunk {chunk_number} to {chunk_filename}')
        offset += chunk_size
        chunk_number += 1

departamentos = [
'AMAZONAS',
'ANTIOQUIA',
'ARAUCA',
 'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA',
 'ARCHIPIELAGO DE SAN ANDRES, PROVIDENCIA Y SANTA CATALINA',
'ARCHIPIÉLAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA',
'ATLANTICO',
 'ATLÁNTICO',
 'BOGOTA',
 'BOGOTA D.C.',
 'BOGOTÁ',
 'BOLIVAR',
 'BOLÍVAR',
'BOYACÁ',
 'CALDAS',
 'CAQUETA',
 'CAQUETÁ',
 'CASANARE',
 'CAUCA',
 'CESAR',
 'CHOCO',
 'CHOCÓ',
 'CORDOBA',
'CUNDINAMARCA',
 'CÓRDOBA',
 'GUAINÍA',
 'GUAVIARE',
 'HUILA',
 'LA GUAJIRA',
 'MAGDALENA',
 'META',
 'NARINO',
 'NARIÑO',
 'NORTE DE SANTANDER',
 'PUTUMAYO',
 'QUINDÍO',
 'RISARALDA',
 'SAN ANDRÉS PROVIDENCIA',
 'SANTANDER',
 'SUCRE',
 'TOLIMA',
 'VALLE DEL CAUCA',
 'VICHADA'
 ]

sleeptime = 1
chunk = 300000
carpeta = 'LluviaPorDepartamento'
idArchivo = "s54a-sgyg"

def descarga_periodica(chunksize, i = 0):
  try:
    download_and_save_data_chunks_departamento(idArchivo, f"/content/drive/My Drive/DANE/lluvia{departamentos[i]}.csv",departamento=departamentos[i],tipo=carpeta,limit=203761118, chunk_size=chunksize)
  except ReadTimeout:
    i = (i+1)%len(departamentos)
    display(Markdown(f'_ReadTimeout, esperando {sleeptime} segundines_'))
    print('')
    time.sleep(sleeptime)
    descarga_periodica(chunksize,i)
descarga_periodica(chunk)

In [ ]:

def concatenate_chunks(filename,carpeta):
  chunk_number = 1
  all_chunks = []
  path = f"/content/drive/My Drive/DANE/chunks/{carpeta}/"
  archive = filename.split('/')[-1][:-4]
  print(f'Now Concatenating {archive}')
  while os.path.exists(f'{path}{archive}/{archive}_chunk_{chunk_number}.csv'):
    print(f'⏳ Concatenating chunk {chunk_number}')
    try:
      all_chunks.append(pd.read_csv(f'{path}{archive}/{archive}_chunk_{chunk_number}.csv'))
      chunk_number += 1
    except Exception as e:
      print(f"Error leyendo chunk {chunk_number}: {e}")
      break
  if all_chunks:
    print(f'Concatenating...')
    final_df = pd.concat(all_chunks, ignore_index=True)
    final_df.to_csv(filename, index=False)
    print(f'💾 Toddos los chunks concatenados en {filename}')
    for i in range(1, chunk_number):
      try:
        os.remove(f'{path}{archive}/{archive}_chunk_{i}.csv')
      except Exception as e:
        print(f"Error rermoviendo chunk {i}: {e}")
    print("Chunk removido.")


In [ ]:
carpeta = 'LluviaPorDepartamento'

for departamento in departamentos:
  concatenate_chunks(f"/content/drive/My Drive/DANE/CSVs listos/vel/vel{departamento}.csv",carpeta=carpeta)